In [80]:
import numpy as np
A = np.eye(10) + np.eye(10, k=1)*2 + np.eye(10, k=-1)*0.5
b = np.ones(10)
x = np.dot(np.linalg.inv(A), b)
print("Ax=b")
print("行列 A")
print(A)
print("右辺 b")
print(b)
print("解 x")
print(x)

Ax=b
行列 A
[[1.  2.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.5 1.  2.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 1.  2.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 1.  2.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 1.  2.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.5 1.  2.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 1.  2.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.5 1.  2.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.5 1.  2. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.5 1. ]]
右辺 b
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
解 x
[ 2.93000000e+02 -1.46000000e+02  2.50000000e-01  3.68750000e+01
 -1.80000000e+01  2.81250000e-01  4.85937500e+00 -2.00000000e+00
  2.85156250e-01  8.57421875e-01]


# BiCG Method

In [81]:
# Initialize solution x
x_bicg = np.ones(len(x))
A_temp = A.copy()
b_base = b.copy()
b_temp = np.dot(A_temp, x_bicg)
r = b_base - b_temp
p = r.copy()
ps = r.copy()
rs = r.copy()
r0 = np.sqrt(np.dot(r, r))
eps = 10 ** -10
for i in range(10000):
    # q _k= Ap_k
    b_temp = np.dot(A_temp, p)
    # q* _k= AHp*_k
    b_temp2 = np.dot(np.linalg.inv(A_temp), ps)
    # (r*_k,r_k)
    t = np.dot(rs, r)
    # alpha_k = (r*_k,r_k) / (p*_k,q_k)
    alpha = t / np.dot(ps, b_temp)
    # x_k1 = x_k + alpha_k * p_k
    x_bicg += alpha * p
    # r_k1 = r_k - alpha_k * q_k
    r -= alpha * b_temp
    # r*_k1 = r*_k - alpha_k * q*_k
    rs -= alpha * b_temp2
    # (r*_k1, r_k1) / (r*_k, r_k)
    beta = np.dot(rs, r) / t
    # p_k1 = r_k1 + beta_k * p_k
    p = r + beta * p
    # p*_k1 = r*_k1 + beta_k * p*_k
    ps = rs + beta * ps
    if i % 1000 == 0:
        print(i, np.sum(x_bicg - x))
    if np.sqrt(np.dot(r, r)) / r0 <= eps:
        print(i, x_bicg - x)
        break
print(np.dot(A, x_bicg))

0 -167.21782655596235
1000 -5.285692407026592e+17
2000 -7.721045015845441e+17
3000 nan
4000 nan


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: divide by zero encountered in double_scalars


5000 nan
6000 nan
7000 nan
8000 nan
9000 nan
[nan nan nan nan nan nan nan nan nan nan]


# [BiCGstab method](https://qiita.com/fukuroder/items/4b708524783192fc2018)

In [82]:
# Initialize solution x
x_bicgstab = np.ones(len(x))
A_temp = A.copy()
b_base = b.copy()
b_temp = np.dot(A_temp, x_bicgstab)
r = b_base - b_temp
p = r.copy()
rs = r.copy()
r0 = np.sqrt(np.dot(r, r))
eps = 10 ** -10
for i in range(10000):
    # q _k= Ap_k
    b_temp = np.dot(A_temp, p)
    # (r*_k,r_k)
    t = np.dot(rs, r)
    # alpha_k = (r*_k,r_k) / (p*_k,q_k)
    alpha = t / np.dot(rs, b_temp)
    # s_k = r_k - alpha_k * q_k
    s = r - alpha * b_temp
    # w_k = (As_k, s_k) / (As_k, As_k)
    As = np.dot(A, s)
    w = np.dot(As, s) / np.dot(As, As)
    # x_k1 = x_k + alpha_k * p_k
    x_bicgstab += alpha * p + w*s
    # r_k1 = s_k - w_k * As_k
    r = s - w * As
    # (r*_k, r_k1) / (r*_k, r_k) * alpha_k / w_k
    beta = np.dot(rs, r) / t * alpha / w
    # p_k1 = r_k1 + beta_k * (p_k - w_k * Ap_k)
    p = r + beta * (p - w * np.dot(A, p)) 
    if i % 1000 == 0:
        print(i, np.sum(x_bicgstab - x))
    if np.sqrt(np.dot(r, r)) / r0 <= eps:
        print(i, x_bicgstab - x)
        print("residual: ", np.sqrt(np.dot(r, r)))
        break
print(np.dot(A, x_bicgstab))

0 -167.20930435246737
18 [ 8.82954510e-09 -4.41056613e-09 -1.08182074e-11  1.11250387e-09
 -5.53971091e-10  1.10272902e-12  1.27274191e-10 -6.17257356e-11
  3.28246319e-11 -2.61596300e-11]
residual:  9.025101595728043e-11
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
